# REINFORCE : Monte-Carlo Policy Gradient

![좋은거](https://leimao.github.io/images/article/2017-05-04-REINFORCE-Policy-Gradient/Sutton_REINFORCE.png)

In [1]:
%%time
## 약 25초 ~30초 소요
!pip install pyvirtualdisplay 
!apt-get install -y xvfb python-opengl ffmpeg
!pip install gym
!pip install box2d-py
#!pip install pyglet==1.3.2
!pip install pyglet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl xvfb
0 upgraded, 2 newly installed, 0 to remove and 13 not upgraded.
Need to get 1,280 kB of archives.
After this operation, 7,686 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 1,280 kB in 3s (501 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 149406 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Selecting previously unselected package xvfb.
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.8_amd64

In [2]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

In [3]:
display = Display(visible=0, size=(1400, 900))
display.start()

In [4]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[-1]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Acrobot-v1

In [5]:
env = wrap_env(gym.make("Acrobot-v1"))
print('observation space:', env.observation_space)
print('action space:', env.action_space)

state = env.reset()
for t in range(1000):
    action = env.action_space.sample() # your agent here (this takes random actions)
    env.render()
    observation, reward, done, info = env.step(action)
    if done: 
      break;
            
print('steps: ', t)
env.close()
show_video()

observation space: Box(-28.274333953857422, 28.274333953857422, (6,), float32)
action space: Discrete(3)
steps:  499


In [6]:
print("state 수는? : ", env.observation_space.shape)
print("action 수는? : ", env.action_space.n)

state 수는? :  (6,)
action 수는? :  3


In [7]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Input, Dense, Add, Subtract, Average, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import plot_model

# Policy Network

1. 히든레이어는 자유롭게 구성하시오.
    * 너무 과하거나 모자라면 엄청 오래걸림.
    * 32, 32, 32 를 시작으로 가감
2. policy(a|s)를 구현하면 됨
    * 인풋이 state
    * 아웃풋은 액션별 확률 ('softmax')

In [8]:
cce = tf.keras.losses.CategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.SUM)

In [9]:
keras.backend.clear_session()
######################
### Your Code Here ###
######################

state_layer = Input(shape=(6,))
hidden = Dense(128, activation='relu')(state_layer)
hidden = Dense(128, activation='relu')(hidden)
policy_layer = Dense(3, activation='softmax')(hidden)


policy_network = Model(state_layer, policy_layer)

policy_network.compile(loss = cce,
              optimizer = Adam(0.01))

policy_network.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
dense (Dense)                (None, 128)               896       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 17,795
Trainable params: 17,795
Non-trainable params: 0
_________________________________________________________________


# Policy를 1회 학습시켜보자.

### 1. policy network를 이용하여 한번의 episode를 기록한다.

In [10]:
state_history = []
action_history = []
reward_history = []

for i in range(1):
    env = wrap_env(gym.make("Acrobot-v1"))
    s0 = env.reset()
    done = False

    cum_r = 0
    time_step = 0
    while True :
        time_step = time_step + 1

        s0 = s0.reshape([1, -1])
        policy = policy_network.predict(s0).reshape(-1)
        a0 = np.random.choice(env.action_space.n, p = policy)
        s1, r1, done, _ = env.step(a0)
        
        a0 = np.eye(env.action_space.n)[a0].reshape([1,-1])
        
        state_history.append(s0)
        action_history.append(a0)
        reward_history.append(r1)

        if time_step % 50 == 0 :
            print(" TimeStep : {} 진행중".format(time_step))

        if done == True :
            env.close()
            show_video()
            break
        s0 = s1

 TimeStep : 50 진행중
 TimeStep : 100 진행중
 TimeStep : 150 진행중
 TimeStep : 200 진행중
 TimeStep : 250 진행중
 TimeStep : 300 진행중
 TimeStep : 350 진행중
 TimeStep : 400 진행중
 TimeStep : 450 진행중
 TimeStep : 500 진행중


# 학습을 진행!

# Return Normalization!

In [11]:
gamma = 0.99  # Discount factor for past rewards
num_inputs = env.observation_space.shape[0]
num_actions = env.action_space.n
eps = np.finfo(np.float32).eps.item()  ## 매우 작은 수.

In [12]:
returns = []
Gt = 0
for r in reward_history[::-1] : # 거꾸로 뽑는다.
    Gt = r + gamma*Gt
    returns.insert(0,  Gt) # append를 0번 인덱스부터 하는 것!

In [13]:
# Normalize
returns = np.array(returns)
returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
returns = returns.reshape([-1,1])
returns.shape

(500, 1)

In [14]:
state = np.vstack(state_history)
state.shape

(500, 6)

In [15]:
action_dummy = np.vstack(action_history)
action_dummy.shape

(500, 3)

In [16]:
G_target = action_dummy*returns

In [17]:
policy_network.fit(state, G_target, verbose=0, epochs=1, batch_size=100000)

# Episode 1000회 학습을 시켜보자!

+ 10개의 최근 에피소드를 모아서 학습을 시켜보자.

In [18]:
from collections import deque
memory = deque(maxlen=2)

In [19]:
n_episode = 1000
gamma = 0.99
rewards_cum = []

for episode in range(n_episode):

    r_c = 0
    #### 에피소드 생성
    state_history = []
    action_history = []
    reward_history = []

    for i in range(1):
        env = wrap_env(gym.make("Acrobot-v1"))
        s0 = env.reset()
        done = False

        time_step = 0
        while True :
            time_step = time_step + 1

            s0 = s0.reshape([1, -1])
            policy = policy_network.predict(s0).reshape(-1)
            a0 = np.random.choice(env.action_space.n, p = policy)
            s1, r1, done, _ = env.step(a0)
            
            a0 = np.eye(env.action_space.n)[a0].reshape([1,-1])
            
            state_history.append(s0)
            action_history.append(a0)
            reward_history.append(r1)

            # if time_step % 100 == 0 :
            #     print(" TimeStep : {} 진행중".format(time_step))

            if done == True :
                env.close()
                # show_video()
                break
            s0 = s1
    rewards_cum.append(sum(reward_history))

    #### 학습
    memory.append((state_history, action_history, reward_history))
    for history in memory :
        state_history = history[0]
        action_history = history[1]
        reward_history = history[2]

        returns = []
        Gt = 0
        for r in reward_history[::-1] : # 거꾸로 뽑는다.
            Gt = r + gamma*Gt
            returns.insert(0,  Gt) # append를 0번 인덱스부터 하는 것!
        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.reshape([-1,1])
        returns.shape

        state = np.vstack(state_history)
        action_dummy = np.vstack(action_history)
        G_target = action_dummy*returns
        policy_network.fit(state, G_target, verbose=0, epochs=1, batch_size=100000)


    if (episode+1) % 2 == 0 :
        print("{}번째 episode 학습 완료".format(episode+1))
        show_video()
        plt.plot(rewards_cum)
        plt.show()


Output hidden; open in https://colab.research.google.com to view.